# **Env Setup**


In [39]:


# Mount google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=False)


# create folder to write data to
#CIS545_FOLDER=os.path.join(DRIVE_MOUNT, 'My Drive', 'CIS545_2020')
#HOMEWORK_FOLDER=os.path.join(CIS545_FOLDER, 'Project')
#os.makedirs(HOMEWORK_FOLDER, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls '/content/drive/Shared drives/1DeepContextGraph/1DeepContextGraph'

code  documents


In [40]:
file_path = '/content/drive/Shared drives/1DeepContextGraph/1DeepContextGraph/code/data/'

# Library

In [41]:
import json
import os
# For caculating approximate time to process notebook (IGNORE)
import datetime
datetime.datetime.now()
import argparse
import pandas as pd
import numpy as np
import operator
import pickle as pkl 
import matplotlib.pyplot as plt
import nltk as nl
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import ParameterGrid
import statistics
import random
import warnings
from string import punctuation
from matplotlib import pyplot
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
import nltk
import re
import io
import requests
import time
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import nltk.sentiment

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


True

# Load Data

In [ ]:
data_fox_claim = pd.read_csv(file_path+'/fox_claim_news_processed.csv',low_memory=False)
#data_all_news = pd.read_csv(file_path+'/statement_checker_all.csv',low_memory=False)
data_polifact = pd.read_csv(file_path+'/political_fact_checker_processed.csv',low_memory=False)


In [57]:
data_fox_claim.columns

Index(['documentId', 'Unnamed: 0.1', 'link', 'published', 'title', 'text',
       'site', 'text_distilled', 'topics', 'named_entity_recognition',
       'top_10_freq_word', 'pasts_of_speech', 'most_similar', 'similar_claims',
       'similar_ratings', 'similar_topics', 'similar_text',
       'similar_text_distilled'],
      dtype='object')

In [ ]:
data_polifact.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'sources', 'urls', 'title', 'summaries',
       'dates', 'ratings', 'text', 'authors', 'publishdate', 'relatedlinks',
       'claims', 'text_distilled', 'topics', 'named_entity_recognition',
       'top_10_freq_word', 'pasts_of_speech'],
      dtype='object')

In [ ]:
data_fox_claim['Unnamed: 0'][1]

1

In [ ]:
data_fox_claim = data_fox_claim.rename({'Unnamed: 0': 'documentId'}, axis='columns')
data_polifact = data_polifact.rename({'Unnamed: 0': 'documentId'}, axis='columns')

# Methods

## Lemmatization
  
First, the raw words must be converted to root forms.  

In [ ]:
def lemmatize(tokenized_words):
  text = [nltk.WordNetLemmatizer().lemmatize(word) for word in tokenized_words]
  return text


## Removing Stop words



In [ ]:
english_stopwords = set(stopwords.words('english') + list(punctuation) + [''])

def remove_stopwords(tokenized_words):
  text = [word for word in tokenized_words if word not in english_stopwords]
  return text


## Custom Filtering

Some of the one or two-letter words from the tokenized words are also removed to further cleanse the raw text. 

In [ ]:
whitelist = set(['ai', 'ax', 'ca', 'eu', 'go', 'io', 'la', 'ox', 'us', 'uk', 
                 'al', 'ak', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'fl', 'ga', 'hi', 
                 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md', 'ma', 'mi',
                 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 'ny',
                 'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn',
                 'tx', 'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy' ])
def remove_too_short(tokenized_words):
  text = [word for word in tokenized_words if (len(word) >= 3 or word not in whitelist) ]
  return text

## LDA Analysis


In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary

def topics(tokenized_words):
    d = Dictionary([tokenized_words])
    c = [d.doc2bow(tokenized_words)]
    m = LdaModel(c, num_topics=1, id2word=d)
    return list(m.print_topics(num_words=2))
  


## Topics as Simple List of Words

A list of topic terms is compiled as show below. The coefficients in front of each word are dropped as part of simplification. The assumption is that the top two words comprising the topic, are both significant enough to be treated equally. It is important that the goal is to build a reliable prediction model. While there is a risk of oversimplification, if the final model results in a poor accuracy score, the coefficient can always be reintroduced here.

In [ ]:
def parseTopics(topics):
   output = []
   words = topics[0][1].split( '+' )
   for word in words:
       output.append( word.split('*')[1].replace( '"', '' ) )
   return output

# 1.Training a Doc2Vec and Word2Vec Model with Gensim

* http://yaronvazana.com/2018/01/20/training-doc2vec-model-with-gensim/
* https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5

## Methods

In [ ]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

class MyCorpus():
    def __init__(self, train_data):
        self.train_data = train_data
        
    def __iter__(self):
        p = PorterStemmer()
        for i in range(len(self.train_data)):
            doc = self.train_data['text'][i]
            doc = re.sub(r'\S*@\S*\s?', '', doc, flags=re.MULTILINE) # remove email
            doc = re.sub(r'http\S+', '', doc, flags=re.MULTILINE) # remove web addresses
            doc = re.sub("\'", "", doc) # remove single quotes
            doc = remove_stopwords(doc)
            doc = p.stem_sentence(doc)
            words = simple_preprocess(doc, deacc=True)
            yield TaggedDocument(words=words, tags=[self.train_data['documentId'][i]])

In [ ]:
class Doc2VecTrainer(object):
    def __init__(self, train_corpus):
        self.train_corpus = train_corpus

    def run(self):
        print('app started')
        model_dimensions = 100
        #cores = multiprocessing.cpu_count()
        #print('num of cores is %s' % cores)
        #gc.collect()
        # if load_existing:
        #     print('loading an exiting model')
        #     model = Doc2Vec.load(PATH_TO_EXISTING_MODEL)
        # else:
        print('reading training corpus from %s' % self.train_corpus)
        corpus_data = MyCorpus(self.train_corpus)
            
        model = Doc2Vec(size=model_dimensions, window=10, min_count=3, sample=1e-4, negative=5,  dm=1) #workers=cores,
        print('building vocabulary...')
        model.build_vocab(corpus_data)

        model.train(corpus_data, total_examples=model.corpus_count, epochs=100)
            
        model.save(file_path+"polifact_doc2vec_model.model")
        model.save_word2vec_format(file_path+"polifact_word2vec_model.model")

        #word2vect
        #model_word2vec = Word2Vec(corpus_data,size=150, window=10,min_count=2,workers=10,iter=10)
        #model.save_word2vec_format(file_path+"polifact_word2vec_model.model")
        #model_word2vec.save(file_path+"polifact_word2vec_model.model")

        print('total docs learned %s' % (len(model.docvecs)))

## Training-Doc2Vec and Word2Vec

In [ ]:
obj = Doc2VecTrainer(data_polifact)
obj.run()

app started
reading training corpus from     documentId  ...                                    pasts_of_speech
0            0  ...  [('Former', 'NNP'), ('President', 'NNP'), ('Do...
1            1  ...  [('Former', 'NNP'), ('President', 'NNP'), ('Do...
2            2  ...  [('Asked', 'VBN'), ('interview', 'NN'), ('Jan....
3            3  ...  [('Former', 'NNP'), ('President', 'NNP'), ('Do...
4            4  ...  [('Former', 'NNP'), ('President', 'NNP'), ('Do...
5            5  ...  [('Throughout', 'IN'), ('president-elect', 'JJ...
6            6  ...  [('A', 'DT'), ('mob', 'NN'), ('breached', 'VBD...
7            7  ...  [('President', 'NNP'), ('Donald', 'NNP'), ('Tr...
8            8  ...  [('When', 'WRB'), ('Congress', 'NNP'), ('meets...
9            9  ...  [('President', 'NNP'), ('Donald', 'NNP'), ('Tr...
10          10  ...  [('In', 'IN'), ('hour-long', 'JJ'), ('phone', ...
11          11  ...  [('President', 'NNP'), ('Donald', 'NNP'), ('Tr...
12          12  ...  [('In', 'IN'), 

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


total docs learned 30


## 1.1Compare Similarity- Using Doc2Vec
https://datascience.stackexchange.com/questions/23969/sentence-similarity-prediction

In [ ]:
from gensim.models.doc2vec import Doc2Vec 

#example for test
model = Doc2Vec.load(file_path+"polifact_doc2vec_model.model")  
new_sentence = "I opened a new mailbox".split(" ")  
model.docvecs.most_similar(positive=[model.infer_vector(new_sentence)],topn=1)

[(7, 0.6654379367828369)]

### Compute

In [ ]:
#try to find most similar claim from politifact for the current fox claim
data_fox_claim['most_similar'] = data_fox_claim['text'].apply(lambda x : model.docvecs.most_similar(positive=[model.infer_vector(re.split('\W+', str(x).lower()))],topn=1)) 

In [ ]:
#text_distilled: try to find most similar claim from politifact for the current fox claim
data_fox_claim['most_similar'] = data_fox_claim['text_distilled'].apply(lambda x : model.docvecs.most_similar(positive=[model.infer_vector(x)],topn=1)) 

### Extract the similar rows from Politifact

In [ ]:
data_polifact.head(2)

,documentId,Unnamed: 0.1,sources,urls,title,summaries,dates,ratings,text,authors,publishdate,relatedlinks,claims,text_distilled,topics,named_entity_recognition,top_10_freq_word,pasts_of_speech
0,0,0,politifact,http://www.politifact.com/factchecks/2021/mar/...,\nSays Deborah Birx “traveled a great distance...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2021-03-29,half-true,\nFormer President Donald Trump didn’t appreci...,Jon Greenberg,2021-03-30,['https://www.cnn.com/2021/03/29/health/pandem...,"National,Coronavirus,Donald Trump","['say', 'deborah', 'birx', 'traveled', 'great'...","['birx ', 'said']","[(Donald, 'PERSON'), (Trump, 'PERSON'), (Biden...","[('birx', 22), ('said', 12), ('family', 10), (...","[('Former', 'NNP'), ('President', 'NNP'), ('Do..."
1,1,1,politifact,http://www.politifact.com/factchecks/2021/mar/...,"\n""We proudly handed the Biden administration ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2021-03-21,barely-true,\nFormer President Donald Trump said that the ...,Amy Sherman,2021-03-24,['https://twitter.com/seanspicer/status/137375...,"Immigration,National,Donald Trump","['proudly', 'handed', 'biden', 'administration...","['border ', 'trump']","[(Donald, 'PERSON'), (Trump, 'PERSON'), (Biden...","[('border', 43), ('trump', 24), ('biden', 16),...","[('Former', 'NNP'), ('President', 'NNP'), ('Do..."


In [ ]:
data_polifact.columns

Index(['documentId', 'Unnamed: 0.1', 'sources', 'urls', 'title', 'summaries',
       'dates', 'ratings', 'text', 'authors', 'publishdate', 'relatedlinks',
       'claims', 'text_distilled', 'topics', 'named_entity_recognition',
       'top_10_freq_word', 'pasts_of_speech'],
      dtype='object')

In [ ]:
for index, row in data_fox_claim.iterrows():
    #print(row['most_similar'], row['most_similar'][0])
    #print("id", row['most_similar'][0][0])
    id = row['most_similar'][0][0]
    data_fox_claim['similar_claims']=data_polifact.iloc[id]['claims']
    data_fox_claim['similar_ratings']=data_polifact.iloc[id]['ratings']
    data_fox_claim['similar_topics']=data_polifact.iloc[id]['topics']
    data_fox_claim['similar_text']=data_polifact.iloc[id]['text']
    data_fox_claim['similar_text_distilled']=data_polifact.iloc[id]['text_distilled']

In [ ]:
data_fox_claim.iloc[[7]]

,documentId,Unnamed: 0.1,link,published,title,text,site,text_distilled,topics,named_entity_recognition,top_10_freq_word,pasts_of_speech,most_similar,similar_claims,similar_ratings,similar_topics,similar_text,similar_text_distilled
7,7,7,https://www.foxnews.com/politics/ex-trump-offi...,2021-04-15T22:09:32,Ex-Trump official Kellyanne Conway named senio...,Former Trump campaign manager and top White Ho...,foxnews,"['ex', 'trump', 'official', 'kellyanne', 'conw...","['trump ', 'moreno']","[(Raphael, 'PERSON'), (Warnock, 'PERSON'), (re...","[('trump', 9), ('moreno', 8), ('campaign', 6),...","[('Former', 'JJ'), ('Trump', 'NNP'), ('campaig...","[(28, 0.12165184319019318)]","Immigration,Homeland Security,Donald Trump",false,"['deportation ', 'biden']",\nFormer President Donald Trump criticized Pre...,"['say', 'joe', 'biden', 'ha', 'effectively', '..."


In [ ]:
print("claim title=",data_fox_claim['title'][7])
print("most_similar=",data_fox_claim['most_similar'][7])
print("similar_claims=",data_fox_claim['similar_claims'][7])
print("similar_ratings=",data_fox_claim['similar_ratings'][7])
print("similar_topics=",data_fox_claim['similar_topics'][7])
print("similar_text_distilled=",data_fox_claim['similar_text_distilled'][7])
print("similar_text=",data_fox_claim['similar_text'][7])

claim title= Ex-Trump official Kellyanne Conway named senior adviser to Ohio Senate candidate Bernie Moreno
most_similar= [(28, 0.12165184319019318)]
similar_claims= Immigration,Homeland Security,Donald Trump
similar_ratings= false
similar_topics= ['deportation ', 'biden']
similar_text_distilled= ['say', 'joe', 'biden', 'ha', 'effectively', 'ordered', 'shutdown', 'ice', 'halting', 'virtually', 'deportation', 'everyone', 'murderer', 'everybody', 'former', 'president', 'donald', 'trump', 'criticized', 'president', 'joe', 'biden', 'reversing', 'trump', 'era', 'immigration', 'policy', 'claimed', 'biden', 'crippled', 'federal', 'agency', 'responsible', 'deportation', 'ha', 'effectively', 'ordered', 'shutdown', 'ice', 'halting', 'virtually', 'deportation', 'everyone', 'murderer', 'everybody', 'trump', 'said', 'feb', '28', 'annual', 'conservative', 'political', 'action', 'conference', 'let', 'deport', 'people', 'restricting', 'law', 'enforcement', 'professional', 'biden', 'revoked', 'several'

### To CSV

In [ ]:
data_fox_claim.head(2)

,documentId,Unnamed: 0.1,link,published,title,text,site,text_distilled,topics,named_entity_recognition,top_10_freq_word,pasts_of_speech,most_similar,similar_claims,similar_ratings,similar_topics,similar_text,similar_text_distilled
0,0,0,https://www.foxnews.com/us/violent-crime-los-a...,2021-04-15T22:33:38,"Murders in Los Angeles up nearly 200%, violent...",Murders in Los Angeles County have spiked near...,foxnews,"['murder', 'los', 'angeles', 'nearly', '200', ...","['angeles ', 'crime']","[(Raphael, 'PERSON'), (Warnock, 'PERSON'), (re...","[('los', 5), ('angeles', 5), ('crime', 5), ('f...","[('Murders', 'NNS'), ('Los', 'NNP'), ('Angeles...","[(27, 0.20057392120361328)]","Immigration,Homeland Security,Donald Trump",false,"['deportation ', 'biden']",\nFormer President Donald Trump criticized Pre...,"['say', 'joe', 'biden', 'ha', 'effectively', '..."
1,1,1,https://www.foxnews.com/politics/raphael-warno...,2021-04-15T22:30:47,Warnock pushes gun control but paid over $100K...,Sen. Raphael Warnock has been calling for gun ...,foxnews,"['warnock', 'push', 'gun', 'control', 'paid', ...","['gun ', 'warnock']","[(Raphael, 'PERSON'), (Warnock, 'PERSON'), (re...","[('warnock', 15), ('gun', 15), ('control', 9),...","[('Sen.', 'NNP'), ('Raphael', 'NNP'), ('Warnoc...","[(23, 0.22679316997528076)]","Immigration,Homeland Security,Donald Trump",false,"['deportation ', 'biden']",\nFormer President Donald Trump criticized Pre...,"['say', 'joe', 'biden', 'ha', 'effectively', '..."


In [ ]:
#data_all_news.to_csv(file_path+'/statement_checker_all_processed.csv')
data_fox_claim.to_csv(file_path+'/fox_claim_news_with_doc2vec_similarity.csv')

## 1.2Compare Similarity- Using Word2Vec

* https://stackoverflow.com/questions/22129943/how-to-calculate-the-sentence-similarity-using-word2vec-model-of-gensim-with-pyt
* https://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.YHvy0OhKhPY

In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from scipy import spatial


In [ ]:
#example for test
model = KeyedVectors.load_word2vec_format(file_path+"polifact_word2vec_model.model")  
index2word_set = set(model.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split(" ")
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#example practice
s1_afv = avg_feature_vector("".join(['trump happy']), model=model, num_features=100, index2word_set=index2word_set)
s2_afv = avg_feature_vector("".join(['hey no']), model=model, num_features=100, index2word_set=index2word_set)
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print("sim=", sim)

sim= 0.7214217782020569


In [ ]:
#example practice
new_word = "trump".split(" ")  
model.wv.most_similar(positive=new_word,topn=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('tweet', 0.9050256609916687)]

In [ ]:
#example practice
s1 = 'trump'
s2 = 'trump' #corrected variable name

distance = model.wv.n_similarity(s1.lower().split(), s2.lower().split())
print("distance=",distance)

distance= 0.99999994


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [ ]:
data_fox_claim_w2v = data_fox_claim

### Computation with AVG score

In [ ]:
import math
similarity_list = []
for index1, row1 in data_fox_claim.iterrows():
    #id = row['most_similar'][0][0]
    max_sim = (-math.inf)
    max_index = 0
    text = "".join(row1['text_distilled'])
    #print("join(row1['text_distilled']=" , text)
    s1_afv = avg_feature_vector(text, model=model, num_features=100, index2word_set=index2word_set)
    #print("s1_afv=",s1_afv)
    for index2, row2 in data_polifact.iterrows():
      #print("index2=", index2)
      #print("row2=",row2)
      text2 = "".join(row1['text_distilled'])
      #print("join(row2['text_distilled']=" , text2)

      s2_afv = avg_feature_vector(text2, model=model, num_features=100, index2word_set=index2word_set)
      #print("s2_afv=",s2_afv)
      sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
      if sim> max_sim:
        max_sim = sim
        max_index = index2
    similarity_list.append((max_index,max_sim))

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [ ]:
similarity_list

[(0, -inf),
 (0, -inf),
 (0, -inf),
 (0, -inf),
 (0, -inf),
 (0, -inf),
 (0, -inf),
 (0, -inf),
 (0, -inf),
 (0, -inf)]

In [ ]:
#try to find most similar claim from politifact for the current fox claim
data_fox_claim_w2v['most_similar'] = similarity_list

### Extract the similar rows from Politifact

In [ ]:
data_polifact.head(2)

,documentId,Unnamed: 0.1,sources,urls,title,summaries,dates,ratings,text,authors,publishdate,relatedlinks,claims,text_distilled,topics,named_entity_recognition,top_10_freq_word,pasts_of_speech
0,0,0,politifact,http://www.politifact.com/factchecks/2021/mar/...,\nSays Deborah Birx “traveled a great distance...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2021-03-29,half-true,\nFormer President Donald Trump didn’t appreci...,Jon Greenberg,2021-03-30,['https://www.cnn.com/2021/03/29/health/pandem...,"National,Coronavirus,Donald Trump","['say', 'deborah', 'birx', 'traveled', 'great'...","['birx ', 'said']","[(Donald, 'PERSON'), (Trump, 'PERSON'), (Biden...","[('birx', 22), ('said', 12), ('family', 10), (...","[('Former', 'NNP'), ('President', 'NNP'), ('Do..."
1,1,1,politifact,http://www.politifact.com/factchecks/2021/mar/...,"\n""We proudly handed the Biden administration ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2021-03-21,barely-true,\nFormer President Donald Trump said that the ...,Amy Sherman,2021-03-24,['https://twitter.com/seanspicer/status/137375...,"Immigration,National,Donald Trump","['proudly', 'handed', 'biden', 'administration...","['border ', 'trump']","[(Donald, 'PERSON'), (Trump, 'PERSON'), (Biden...","[('border', 43), ('trump', 24), ('biden', 16),...","[('Former', 'NNP'), ('President', 'NNP'), ('Do..."


In [ ]:
data_polifact.columns

Index(['documentId', 'Unnamed: 0.1', 'sources', 'urls', 'title', 'summaries',
       'dates', 'ratings', 'text', 'authors', 'publishdate', 'relatedlinks',
       'claims', 'text_distilled', 'topics', 'named_entity_recognition',
       'top_10_freq_word', 'pasts_of_speech'],
      dtype='object')

In [ ]:
for index, row in data_fox_claim_w2v.iterrows():
    #print(row['most_similar'], row['most_similar'][0])
    #print("id", row['most_similar'][0][0])
    id = row['most_similar'][0]
    data_fox_claim_w2v['similar_claims']=data_polifact.iloc[id]['claims']
    data_fox_claim_w2v['similar_ratings']=data_polifact.iloc[id]['ratings']
    data_fox_claim_w2v['similar_topics']=data_polifact.iloc[id]['topics']
    data_fox_claim_w2v['similar_text']=data_polifact.iloc[id]['text']
    data_fox_claim_w2v['similar_text_distilled']=data_polifact.iloc[id]['text_distilled']

In [ ]:
data_fox_claim_w2v.iloc[[7]]

,documentId,Unnamed: 0.1,link,published,title,text,site,text_distilled,topics,named_entity_recognition,top_10_freq_word,pasts_of_speech,most_similar,similar_claims,similar_ratings,similar_topics,similar_text,similar_text_distilled
7,7,7,https://www.foxnews.com/politics/ex-trump-offi...,2021-04-15T22:09:32,Ex-Trump official Kellyanne Conway named senio...,Former Trump campaign manager and top White Ho...,foxnews,"['ex', 'trump', 'official', 'kellyanne', 'conw...","['trump ', 'moreno']","[(Raphael, 'PERSON'), (Warnock, 'PERSON'), (re...","[('trump', 9), ('moreno', 8), ('campaign', 6),...","[('Former', 'JJ'), ('Trump', 'NNP'), ('campaig...","(0, -inf)","National,Coronavirus,Donald Trump",half-true,"['birx ', 'said']",\nFormer President Donald Trump didn’t appreci...,"['say', 'deborah', 'birx', 'traveled', 'great'..."


In [ ]:
print("claim title=",data_fox_claim_w2v['title'][7])
print("most_similar=",data_fox_claim_w2v['most_similar'][7])
print("similar_claims=",data_fox_claim_w2v['similar_claims'][7])
print("similar_ratings=",data_fox_claim_w2v['similar_ratings'][7])
print("similar_topics=",data_fox_claim_w2v['similar_topics'][7])
print("similar_text_distilled=",data_fox_claim_w2v['similar_text_distilled'][7])
print("similar_text=",data_fox_claim_w2v['similar_text'][7])

claim title= Ex-Trump official Kellyanne Conway named senior adviser to Ohio Senate candidate Bernie Moreno
most_similar= (0, -inf)
similar_claims= National,Coronavirus,Donald Trump
similar_ratings= half-true
similar_topics= ['birx ', 'said']
similar_text_distilled= ['say', 'deborah', 'birx', 'traveled', 'great', 'distance', 'see', 'family', 'thanksgiving', 'call', 'police', 'turn', 'resigned', 'former', 'president', 'donald', 'trump', 'appreciate', 'comment', 'dr', 'deborah', 'birx', 'made', 'cnn', 'report', 'criticizing', 'administration', 'handling', 'coronavirus', 'birx', 'coordinator', 'white', 'house', 'coronavirus', 'task', 'force', 'said', 'trump', 'administration', 'failed', 'give', 'public', 'clear', 'consistent', 'guidance', 'laid', 'many', 'death', 'administration', 'foot', '100', '000', 'death', 'came', 'original', 'surge', 'birx', 'said', 'march', '28', 'broadcast', 'featuring', 'several', 'doctor', 'advised', 'trump', 'rest', 'mind', 'could', 'mitigated', 'decreased', 's

### To CSV

In [ ]:
data_fox_claim_w2v.head(2)

,documentId,Unnamed: 0.1,link,published,title,text,site,text_distilled,topics,named_entity_recognition,top_10_freq_word,pasts_of_speech,most_similar,similar_claims,similar_ratings,similar_topics,similar_text,similar_text_distilled
0,0,0,https://www.foxnews.com/us/violent-crime-los-a...,2021-04-15T22:33:38,"Murders in Los Angeles up nearly 200%, violent...",Murders in Los Angeles County have spiked near...,foxnews,"['murder', 'los', 'angeles', 'nearly', '200', ...","['angeles ', 'crime']","[(Raphael, 'PERSON'), (Warnock, 'PERSON'), (re...","[('los', 5), ('angeles', 5), ('crime', 5), ('f...","[('Murders', 'NNS'), ('Los', 'NNP'), ('Angeles...","(0, -inf)","National,Coronavirus,Donald Trump",half-true,"['birx ', 'said']",\nFormer President Donald Trump didn’t appreci...,"['say', 'deborah', 'birx', 'traveled', 'great'..."
1,1,1,https://www.foxnews.com/politics/raphael-warno...,2021-04-15T22:30:47,Warnock pushes gun control but paid over $100K...,Sen. Raphael Warnock has been calling for gun ...,foxnews,"['warnock', 'push', 'gun', 'control', 'paid', ...","['gun ', 'warnock']","[(Raphael, 'PERSON'), (Warnock, 'PERSON'), (re...","[('warnock', 15), ('gun', 15), ('control', 9),...","[('Sen.', 'NNP'), ('Raphael', 'NNP'), ('Warnoc...","(0, -inf)","National,Coronavirus,Donald Trump",half-true,"['birx ', 'said']",\nFormer President Donald Trump didn’t appreci...,"['say', 'deborah', 'birx', 'traveled', 'great'..."


In [ ]:
#data_all_news.to_csv(file_path+'/statement_checker_all_processed.csv')
data_fox_claim_w2v.to_csv(file_path+'/fox_claim_news_with_word2vec_similarity.csv')

# 2.Compare Similarity- Sentence BERT

## Method-1: S-BERT not working

In [ ]:
!pip install torch==1.2.0 torchvision==0.4.0 -f


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-f option requires 1 argument


In [ ]:
#(from versions: 0.1.0, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.4.1, 0.2.5, 0.2.5.1,
 #0.2.6, 0.2.6.1, 0.2.6.2, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.5, 0.3.5.1, 0.3.6, 0.3.7, 0.3.7.1, 0.3.7.2,
 #0.3.8, 0.3.9, 0.4.0, 0.4.1, 0.4.1.1, 0.4.1.2, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4)

!pip install -U sentence-transformers==0.3.2

     |████████████████████████████████| 71kB 5.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.2-cp37-none-any.whl size=93966 sha256=03c05e47e28d6bfe2ee13e9f8c1b97a81f59cf3d12e9136c3ae6ba01f5348dfa
  Stored in directory: /root/.cache/pip/wheels/f7/06/a0/567f3651876165429f6510d3197b011652a25e547552816824
Successfully built sentence-transformers
  Found existing installation: sentence-transformers 1.0.1
    Uninstalling sentence-transformers-1.0.1:
      Successfully uninstalled sentence-transformers-1.0.1


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:

#model = SentenceTransformer('paraphrase-distilroberta-base-v1')
model = SentenceTransformer('xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

NameError: ignored

# 3.Compute Similarity: DistilBERT model

https://towardsdatascience.com/how-to-compute-sentence-similarity-using-bert-and-word2vec-ab0663a5d64

In [42]:
!pip install sent2vec

In [43]:
from scipy import spatial
from sent2vec.vectorizer import Vectorizer

## Example-Experiment

In [ ]:
sentences = [
 "This is an awesome book to learn NLP.",
 "DistilBERT is an amazing NLP model.",
 "We can interchangeably use embedding, encoding, or vectorizing.",
]
s1 = "I am here. "
s2 = "My name is soma"
s3 = "Go away"

vectorizer = Vectorizer()
vectorizer.bert(s1)
vectors_bert1 = vectorizer.vectors

vectorizer.bert(s2)
vectors_bert2 = vectorizer.vectors

vectorizer.bert(s3)
vectors_bert3 = vectorizer.vectors

In [ ]:
vectors_bert1

array([[-0.00452068, -0.01841036,  0.11396753, ..., -0.03791968,
         0.28177658,  0.26435444],
       [-0.21792541,  0.01430414,  0.12090803, ..., -0.09087957,
         0.0732041 ,  0.20216972],
       [-0.22053696, -0.16753876,  0.092399  , ..., -0.01936097,
         0.08270013,  0.29839826],
       ...,
       [-0.13918301, -0.02728104,  0.22536631, ..., -0.06013088,
         0.18442337,  0.426399  ],
       [-0.15340249,  0.04992858,  0.24422099, ..., -0.00687042,
         0.11672808,  0.3288559 ],
       [-0.22053696, -0.16753876,  0.092399  , ..., -0.01936097,
         0.08270013,  0.29839826]], dtype=float32)

In [ ]:
print(len(vectors_bert1))
print(len(vectors_bert2))
print(len(vectors_bert3))

11
15
7


In [ ]:
dist_1 = spatial.distance.cosine(vectors_bert1[0], vectors_bert2[0])
dist_2 = spatial.distance.cosine(vectors_bert1[0], vectors_bert3[0])
print('sim_1: {0}, sim_2: {1}'.format(1-dist_1, 1-dist_2))
# dist_1: 0.043, dist_2: 0.192

sim_1: 0.9835779666900635, sim_2: 0.9676865339279175


## Compute Similarity Using DistilBERT

In [44]:
data_fox_claim_w2v = data_fox_claim

In [45]:
data_polifact.columns

Index(['documentId', 'Unnamed: 0.1', 'sources', 'urls', 'title', 'summaries',
       'dates', 'ratings', 'text', 'authors', 'publishdate', 'relatedlinks',
       'claims', 'text_distilled', 'topics', 'named_entity_recognition',
       'top_10_freq_word', 'pasts_of_speech'],
      dtype='object')

In [51]:
import math
vectorizer = Vectorizer()


similarity_list = []
i = 0
j= 0
for index1, row1 in data_fox_claim_w2v.iterrows():
    #id = row['most_similar'][0][0]
    max_sim = (-math.inf)
    max_index = -1
    text1 = "".join(row1['text'])
    #print("\njoin(row1['text_distilled']=" , text1)

    #vectorize for bert
    vectorizer.bert(text1)
    vectors_bert1 = vectorizer.vectors
    for index2, row2 in data_polifact.iterrows():
      #if j==1: break
      #print("index2=", index2)
      #print("row2=",row2)
      text2 = "".join(row2['claims'])
      #text2 = ['My', 'name', 'is' ,'soma']
      #print("\njoin(row2['text_distilled']=" , text2)

      #vectorize for bert
      vectorizer.bert(text2)
      vectors_bert2 = vectorizer.vectors

      sim = 1 - spatial.distance.cosine(vectors_bert1[0], vectors_bert2[0])
      #print("\sim=", sim, "index2=", index2)
      if sim> max_sim:
        max_sim = sim
        max_index = index2
      j= 1
    similarity_list.append((max_index,max_sim))
    i = 1
    #if i==1: break

In [52]:
similarity_list

[(12, 1.0),
 (12, 0.9898593425750732),
 (12, 0.990394651889801),
 (14, 1.0),
 (7, 0.9858864545822144),
 (5, 1.0),
 (0, 1.0),
 (7, 0.9874798655509949),
 (5, 1.0),
 (13, 1.0)]

In [53]:
#try to find most similar claim from politifact for the current fox claim
data_fox_claim_w2v['most_similar'] = similarity_list

## Extract the similar rows from Politifact

In [ ]:
data_polifact.head(2)

,documentId,Unnamed: 0.1,sources,urls,title,summaries,dates,ratings,text,authors,publishdate,relatedlinks,claims,text_distilled,topics,named_entity_recognition,top_10_freq_word,pasts_of_speech
0,0,0,politifact,http://www.politifact.com/factchecks/2021/mar/...,\nSays Deborah Birx “traveled a great distance...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2021-03-29,half-true,\nFormer President Donald Trump didn’t appreci...,Jon Greenberg,2021-03-30,['https://www.cnn.com/2021/03/29/health/pandem...,"National,Coronavirus,Donald Trump","['say', 'deborah', 'birx', 'traveled', 'great'...","['birx ', 'said']","[(Donald, 'PERSON'), (Trump, 'PERSON'), (Biden...","[('birx', 22), ('said', 12), ('family', 10), (...","[('Former', 'NNP'), ('President', 'NNP'), ('Do..."
1,1,1,politifact,http://www.politifact.com/factchecks/2021/mar/...,"\n""We proudly handed the Biden administration ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2021-03-21,barely-true,\nFormer President Donald Trump said that the ...,Amy Sherman,2021-03-24,['https://twitter.com/seanspicer/status/137375...,"Immigration,National,Donald Trump","['proudly', 'handed', 'biden', 'administration...","['border ', 'trump']","[(Donald, 'PERSON'), (Trump, 'PERSON'), (Biden...","[('border', 43), ('trump', 24), ('biden', 16),...","[('Former', 'NNP'), ('President', 'NNP'), ('Do..."


In [ ]:
data_polifact.columns

Index(['documentId', 'Unnamed: 0.1', 'sources', 'urls', 'title', 'summaries',
       'dates', 'ratings', 'text', 'authors', 'publishdate', 'relatedlinks',
       'claims', 'text_distilled', 'topics', 'named_entity_recognition',
       'top_10_freq_word', 'pasts_of_speech'],
      dtype='object')

In [54]:
for index, row in data_fox_claim_w2v.iterrows():
    #print(row['most_similar'], row['most_similar'][0])
    #print("id", row['most_similar'][0][0])
    id = row['most_similar'][0]
    data_fox_claim_w2v['similar_claims']=data_polifact.iloc[id]['claims']
    data_fox_claim_w2v['similar_ratings']=data_polifact.iloc[id]['ratings']
    data_fox_claim_w2v['similar_topics']=data_polifact.iloc[id]['topics']
    data_fox_claim_w2v['similar_text']=data_polifact.iloc[id]['text']
    data_fox_claim_w2v['similar_text_distilled']=data_polifact.iloc[id]['text_distilled']

In [ ]:
data_fox_claim_w2v.iloc[[7]]

,documentId,Unnamed: 0.1,link,published,title,text,site,text_distilled,topics,named_entity_recognition,top_10_freq_word,pasts_of_speech,most_similar,similar_claims,similar_ratings,similar_topics,similar_text,similar_text_distilled
7,7,7,https://www.foxnews.com/politics/ex-trump-offi...,2021-04-15T22:09:32,Ex-Trump official Kellyanne Conway named senio...,Former Trump campaign manager and top White Ho...,foxnews,"['ex', 'trump', 'official', 'kellyanne', 'conw...","['trump ', 'moreno']","[(Raphael, 'PERSON'), (Warnock, 'PERSON'), (re...","[('trump', 9), ('moreno', 8), ('campaign', 6),...","[('Former', 'JJ'), ('Trump', 'NNP'), ('campaig...","(0, 0.9588000774383545)","National,Coronavirus,Donald Trump",half-true,"['birx ', 'said']",\nFormer President Donald Trump didn’t appreci...,"['say', 'deborah', 'birx', 'traveled', 'great'..."


In [58]:
print("claim title=",data_fox_claim_w2v['title'][7])
print("claim topics=",data_fox_claim_w2v['topics'][7])
print("most_similar=",data_fox_claim_w2v['most_similar'][7])
print("similar_claims=",data_fox_claim_w2v['similar_claims'][7])
print("similar_ratings=",data_fox_claim_w2v['similar_ratings'][7])
print("similar_topics=",data_fox_claim_w2v['similar_topics'][7])
print("similar_text_distilled=",data_fox_claim_w2v['similar_text_distilled'][7])
print("similar_text=",data_fox_claim_w2v['similar_text'][7])

claim title= Ex-Trump official Kellyanne Conway named senior adviser to Ohio Senate candidate Bernie Moreno
claim topics= ['trump ', 'moreno']
most_similar= (7, 0.9874798655509949)
similar_claims= Public Health,Health Check,Donald Trump
similar_ratings= false
similar_topics= ['covid ', 'immunity']
similar_text_distilled= ['percentage', 'american', 'natural', 'immunity', 'getting', 'covid', '19', 'powerful', 'vaccine', 'dec', '8', 'press', 'conference', 'operation', 'warp', 'speed', 'president', 'donald', 'trump', 'likened', 'spread', 'coronavirus', 'throughout', 'population', 'expert', 'agree', 'bestows', 'immunity', 'people', 'became', 'ill', 'covid', '19', 'vaccine', 'develop', 'immunity', 'period', 'time', 'hear', 'close', '15', 'hearing', 'terrific', 'powerful', 'vaccine', 'said', 'trump', 'responding', 'reporter', 'question', 'message', 'american', 'people', 'holiday', 'approach', 'level', 'covid', 'case', 'u', 'continue', 'rise', 'first', 'time', 'trump', 'given', 'credence', 'id

## To CSV

In [ ]:
data_fox_claim_w2v.head(2)

,documentId,Unnamed: 0.1,link,published,title,text,site,text_distilled,topics,named_entity_recognition,top_10_freq_word,pasts_of_speech,most_similar,similar_claims,similar_ratings,similar_topics,similar_text,similar_text_distilled
0,0,0,https://www.foxnews.com/us/violent-crime-los-a...,2021-04-15T22:33:38,"Murders in Los Angeles up nearly 200%, violent...",Murders in Los Angeles County have spiked near...,foxnews,"['murder', 'los', 'angeles', 'nearly', '200', ...","['angeles ', 'crime']","[(Raphael, 'PERSON'), (Warnock, 'PERSON'), (re...","[('los', 5), ('angeles', 5), ('crime', 5), ('f...","[('Murders', 'NNS'), ('Los', 'NNP'), ('Angeles...","(0, 1.0)","National,Coronavirus,Donald Trump",half-true,"['birx ', 'said']",\nFormer President Donald Trump didn’t appreci...,"['say', 'deborah', 'birx', 'traveled', 'great'..."
1,1,1,https://www.foxnews.com/politics/raphael-warno...,2021-04-15T22:30:47,Warnock pushes gun control but paid over $100K...,Sen. Raphael Warnock has been calling for gun ...,foxnews,"['warnock', 'push', 'gun', 'control', 'paid', ...","['gun ', 'warnock']","[(Raphael, 'PERSON'), (Warnock, 'PERSON'), (re...","[('warnock', 15), ('gun', 15), ('control', 9),...","[('Sen.', 'NNP'), ('Raphael', 'NNP'), ('Warnoc...","(0, 1.0)","National,Coronavirus,Donald Trump",half-true,"['birx ', 'said']",\nFormer President Donald Trump didn’t appreci...,"['say', 'deborah', 'birx', 'traveled', 'great'..."


In [56]:
#data_all_news.to_csv(file_path+'/statement_checker_all_processed.csv')
data_fox_claim_w2v.to_csv(file_path+'/fox_claim_news_with_distilBERT_similarity.csv')